#### Notebook Author: Janghoon Lee
#### Code Revision: Janghoon Lee
#### Algorithm Implemented: A. Fischer and C. Igel: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.727.7087&rep=rep1&type=pdf 
#### Source Code Referred: https://www.superdatascience.com/deep-learning/
####  Movie Data Used:(https://grouplens.org/datasets/movielens/) 

# Architecture Concept
![Concept Image](https://cdn-images-1.medium.com/max/1600/1*ZY4c980_7MfEMYTIi6jvTw.png)
*Figure from https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-i-6df5c4918c15*

# Modules Import

In [100]:
# Boltzmann Machines
# Importing the libraries
import numpy as np
import pandas as pd

In [101]:
# Import PyTorch library required to build Boltzmann Machines
# Open-source code borrowed from https://github.com/pytorch/pytorch
import torch # 
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Movies Data

In [102]:
# Import the movies dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
# Show 10 rows from the top
movies.head(10)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


* Column 0 = Movie ID
* Column 1 = Movie Title
* Column 2 = Movie Category

# User Data

In [103]:
# Import the users dataset
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
# Show 10 rows from the top
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


* Column 0 = User ID
* Column 1 = User Gender (F = Female, M = Male)
* Column 2 = User Age
* Column 3 = User Job (Must link to Jobs.data, but irrelevant for Khan Academy demo)
* Column 4 = User Code (Irrelevant for Khan Academy demo)

# Movie Ratings Data

In [104]:
# Import the ratings dataset
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


* Column 0 = User ID
* Column 1 = Movie ID
* Column 2 = Movie Ratings (1 = Absolutely Not Like, 5 = Absolutely Like)
* Column 3 = Timestamp of rating (irrelevant)

# Training Set

In [105]:
# Preparing the training set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set.head(10)

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
5,1,8,1,875072484
6,1,9,5,878543541
7,1,11,2,875072262
8,1,13,5,875071805
9,1,15,5,875071608


* Column 0 = User ID
* Column 1 = Movie ID
* Column 2 = Movie Ratings (1 = Absolutely Not Like, 5 = Absolutely Like)
* Column 3 = Timestamp of rating (irrelevant)

In [106]:
# Convert the pandas dataframe to array of integer for easier data manipulation
training_set = np.array(training_set, dtype = 'int')
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

# Test Set

In [107]:
# Prepare the test set
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set.head(10)

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883
5,1,23,4,875072895
6,1,24,3,875071713
7,1,27,2,876892946
8,1,31,3,875072144
9,1,33,4,878542699


* Column 0 = User ID
* Column 1 = Movie ID
* Column 2 = Movie Ratings (1 = Absolutely Not Like, 5 = Absolutely Like)
* Column 3 = Timestamp of rating (irrelevant)

In [108]:
# Convert the pandas dataframe to array of integer for easier data manipulation
test_set = np.array(test_set, dtype = 'int')

# Data Processing

In [109]:
# Getting the maximum number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [110]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1): # Do the following for the size of the data
        id_movies = data[:,1][data[:,0] == id_users] # Retrieve movie id column of this user
        id_ratings = data[:,2][data[:,0] == id_users] # Retrieve rating id column of this user
        ratings = np.zeros(nb_movies) # Create a placeholder list of zeros
        ratings[id_movies - 1] = id_ratings # Fill in with actual rating data from 1 to 5
        new_data.append(list(ratings)) # Add the list data to the list
    return new_data # Return the processed list

# Run the convert function
training_set = convert(training_set)
test_set = convert(test_set)

In [111]:
# Converting the data into Torch tensors to allow PyTorch library to take over
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [112]:
# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Restricted Boltzmann Machine (RBM) Architecture Implementation

In [113]:
# Creating the architecture of the Neural Network using class
class RBM():
    # Initialize parameters to be created when the class is instantiated
    # nv = number of visible nodes
    # nh = number of hidden nodes
    def __init__(self, nv, nh):
        # W = weight = probability of visible nodes given the hidden nodes
        self.W = torch.randn(nh, nv) # Initialize randomly according to normal distribution
        # a = bias for the probability of hidden nodes given the visible nodes
        self.a = torch.randn(1, nh) # Initialize P(h | v) with normal distribution
        # b = bias for the probability of visible nodes given the hidden nodes 
        self.b = torch.randn(1, nv)  #Initialize P(v | h) with normal distribution
    
    # Gibbs Sampling hidden nodes according to P(h | v)
    def sample_h(self, x):
        # Multiply weight with v in P(h | v)  
        wx = torch.mm(x, self.W.t())
        # Add wx + bias a for activation
        activation = wx + self.a.expand_as(wx)
        # Calculate the probability of hidden node being 1, given v
        p_h_given_v = torch.sigmoid(activation) # Caculate the sigmoid of the activation value
        # Return the calculated probability and sampled data with the probability
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    # Gibbs Sampling visible nodes according to P(v | h)
    def sample_v(self, y):
        # Multiply weight with h in P(v | h)  
        wy = torch.mm(y, self.W)
        # Add wx + bias b for activation
        activation = wy + self.b.expand_as(wy)
        # Calculate the probability of visible node being 1, given h
        p_v_given_h = torch.sigmoid(activation) # Caculate the sigmoid of the activation value
        # Return the calculated probability and sampled data with the probability
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    
    # v0 = input vector containing ratings of the user
    # vk = visible nodes obtained after k-interations of contrastive convergences
    # ph0 = vector of P(h = 1 | v)
    # phk = vector of P(h | vk) after k-sampling
    def train(self, v0, vk, ph0, phk):
        # Update weights with the difference of (transpoed ph0 * v0 - transposed phk * vk)
        self.W += torch.mm(ph0.t(),v0) - torch.mm(phk.t(),vk)
        # Update bias b with the difference of v0 and vk
        self.b += torch.sum((v0 - vk), 0, )
        # Update bias a with the difference of ph0 and phk
        self.a += torch.sum((ph0 - phk), 0)

# RBM: Parameter Declaration

In [114]:
# Get number of visible nodes from the training set
nv = len(training_set[0])
# Set number of hidden nodes (features to detect) to 100; can be optimized after discussion with Khan Academy
nh = 100
# Set number of batch size per iteration to 100; can be optimized after discussion with Khan Academy
batch_size = 100
# Instantiate RBM class defined above with the parameters
rbm = RBM(nv, nh)

# RBM Training

In [115]:
# Training the RBM

# Run the training for 10 epochs; can be optimized with Khan Academy
nb_epoch = 10
# For the number of epochs, do the following and record the loss of each training
for epoch in range(1, nb_epoch + 1):
    train_loss = 0 # Initialize with 0
    s = 0. # Initialize the counter
    
    # For each user in the batch size, do the following
    for id_user in range(0, nb_users - batch_size, batch_size):
        # Select all the users from current user position to the batch size position as input batch
        vk = training_set[id_user:id_user+batch_size]
        # Select all the users from current user position to the batch size position as comparison batch
        v0 = training_set[id_user:id_user+batch_size]
        # Set P(h | v) of with sample_v function defined in class
        ph0,_ = rbm.sample_h(v0)
        
        # Perform k-contrastive divergence 10 times
        for k in range(10):
            _,hk = rbm.sample_h(vk) # Sample h on visible nodes
            _,vk = rbm.sample_v(hk) # Sample v on hidden nodes
            vk[v0<0] = v0[v0<0] # Ignore cells that contain -1 ratings
            
        # Approximate gradient 
        phk,_ = rbm.sample_h(vk) # Sample h on the last sample of updated visible nodes
        rbm.train(v0, vk, ph0, phk) # Start training with the function defined above
        
        # Update the train loss by taking the difference between predicted rating and actual rating
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1. # Add iteration counter
        
    # Print current epoch's normalized train loss
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3376)
epoch: 2 loss: tensor(0.2512)
epoch: 3 loss: tensor(0.2404)
epoch: 4 loss: tensor(0.2520)
epoch: 5 loss: tensor(0.2486)
epoch: 6 loss: tensor(0.2512)
epoch: 7 loss: tensor(0.2463)
epoch: 8 loss: tensor(0.2469)
epoch: 9 loss: tensor(0.2495)
epoch: 10 loss: tensor(0.2494)


#### Interpretation of the train loss: the demo model wrongly predicted 25% of user's rating when compared to the actual data

# RBM: Testing

In [117]:
# Testing the RBM
test_loss = 0 # Initialize the train loss to 0
s = 0. # Initialize the counter to 0 for test loss normalization

# For each user in the test set, do the following
for id_user in range(nb_users):
    # Set the comparison source set to the predicted ratings from the training set
    v = training_set[id_user:id_user+1]
    # Set the comparison target set to the ratings from the test set
    vt = test_set[id_user:id_user+1]
    
    # For every ratings where the rating is not 0, do the following
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v) # Sample h on visible nodes
        _,v = rbm.sample_v(h) # Sample v on hidden nodes
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # Sample h on the last sample of updated visible nodes
        s += 1. # Add iteration counter
        
# Print the normalized test loss for model evaluation
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2488)


#### Interpretation of the test loss:
* The train loss value is consistent with the test loss value
* The demo model wrongly predicted ~25% of user's rating when compared to the actual data
  * There was 75% was success and it will most likely produce the same accuracy in other datasets